## Lesson 4: Tools for a Customer Outreach Campaign

https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai/lesson/9/tools-for-a-customer-outreach-campaign-(code)

In this lesson you will learn more about tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

In [1]:
import warnings

from crewai import Agent, Crew, Task
from crewai_tools import BaseTool, DirectoryReadTool, FileReadTool, SerperDevTool
from dotenv import find_dotenv, load_dotenv
from IPython.display import Markdown

warnings.filterwarnings("ignore")

In [2]:
load_dotenv(find_dotenv())

True

### Creating Agents

In [3]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match our ideal customer profile",
    backstory=(
        "As part of the dynamic sales team at CrewAI, your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools and a strategic mindset, you analyze "
        "data, trends and interactions to unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way for meaningful engagements and "
        "driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True,
)

In [4]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communication",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients and the solutions they need. "
        "By creating engaging, personalized messages, you not only inform leads about "
        "our offering but also make them feel seen and heard. "
        "Your role is pivotal in converting interest into action, guiding leads "
        "through the journey from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True,
)

### Creating Tools

#### crewAI Tools

In [5]:
directory_read_tool = DirectoryReadTool(directory="./instructions")
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

#### Custom Tool

- Every tool needs to have a `name` and a `description`
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [6]:
class SentimentAnalysisTool(BaseTool):
    name: str = "Sentiment Analysis Tool"
    description: str = (
        "Analyzes the sentiment of text to ensure positive and engaging communication."
    )

    def _run(self, text: str) -> str:
        # custom code goes here
        return "positive"

In [7]:
sentiment_analysis_tool = SentimentAnalysisTool()

### Creating Tasks
- The Lead Profiling Task is using crewAI tools.

In [8]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The personalized outreach task is using your custom tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool`.

In [9]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

#### Creating the Crew

In [10]:
crew = Crew(
    agents=[sales_rep_agent, lead_sales_rep_agent],
    tasks=[lead_profiling_task, personalized_outreach_task],
    verbose=2,
    memory=True,
    # embedder={"provider": "openai", "config": {"model": "embedding-2"}},
    embedder={"provider": "openai"},
)

#### Running the Crew

In [11]:
inputs = {
    "lead_name": "DeepLearning.AI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch",
}

In [12]:
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearning.AI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather all available data on DeepLearning.AI to compile a comprehensive report that will help tailor our engagement strategy effectively.

Action:
Search the internet

Action Input:
{"search_query": "DeepLearning.AI company profile"}
 


Search results: Title: DeepLearning.AI - LinkedIn
Link: https://www.linkedin.com/company/deeplearningai
Snippet: DeepLearning.AI is making a world-class AI edu

In [13]:
Markdown(data=result)

Email Draft to Andrew Ng:

Subject: Congratulations on the Launch of "Generative AI for Everyone"!

Dear Andrew Ng,

I hope this message finds you well. I wanted to extend my heartfelt congratulations to you and the entire DeepLearning.AI team on the recent launch of "Generative AI for Everyone" in collaboration with Coursera. It is truly inspiring to see the continuous innovation and dedication to democratizing AI education on a global scale.

As a global leader in AI education, DeepLearning.AI's commitment to making world-class AI education accessible to people globally aligns perfectly with our mission to provide advanced tools and technologies to enhance the learning experience for users like yourselves. We believe that our personalized training modules tailored to DeepLearning.AI's specific curriculum and target audience can further support your efforts in educating and empowering individuals in the field of AI.

I would like to offer you an exclusive opportunity to experience a demo of our solutions, showcasing how they can seamlessly integrate with DeepLearning.AI's platform and enhance the overall user experience. Additionally, we have compelling case studies of successful collaborations with organizations in the online learning sector that demonstrate the value our solutions can bring to DeepLearning.AI.

I am confident that our innovative solutions can complement DeepLearning.AI's existing educational platform and contribute to your mission of expanding reach and impact in the AI education sector. I would be thrilled to discuss further how we can support your goals and enhance the educational offerings of DeepLearning.AI.

Thank you for your time, Andrew. I look forward to the opportunity to connect and explore how our solutions can benefit DeepLearning.AI.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

---

Please adjust the email draft to include your actual name before sending it out to Andrew Ng.